# Twitter Data Scrape
This notebook focuses on scraping user profile data using user id and tag.
The idea is to collect a list of user id associated with certain tag and collect information from the user profile using the list of user id. 

In [1]:
# Import libraries
from ntscraper import Nitter
import pandas as pd
from pprint import pprint

11-Sep-24 18:03:23 - Note: NumExpr detected 22 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
11-Sep-24 18:03:23 - NumExpr defaulting to 8 threads.


### Data Scrape using Nitter

In [4]:
# Set up scraper
scraper = Nitter(log_level=1, skip_instance_check=False)

# Extract user id associated with certain hashtag
twitter_hash_tweets = scraper.get_tweets("PresidentialDebate2024", mode = "hashtag", number = 10)

Testing instances: 100%|██████████| 16/16 [00:20<00:00,  1.30s/it]

11-Sep-24 18:17:14 - No instance specified, using random instance https://nitter.lucabased.xyz


11-Sep-24 18:17:26 - Current stats for PresidentialDebate2024: 9 tweets, 0 threads...
11-Sep-24 18:17:37 - Current stats for PresidentialDebate2024: 10 tweets, 0 threads...


In [5]:
pprint(twitter_hash_tweets)

{'threads': [],
 'tweets': [{'date': 'Sep 11, 2024 · 2:36 AM UTC',
             'external-link': '',
             'gifs': [],
             'is-pinned': False,
             'is-retweet': True,
             'link': 'https://twitter.com/tmprowell/status/1833696208516178170#m',
             'pictures': [],
             'quoted-post': {},
             'replying-to': [],
             'stats': {'comments': 21,
                       'likes': 2309,
                       'quotes': 20,
                       'retweets': 667},
             'text': 'I was a dr before the Affordable Care Act. And people w/ '
                     'cancer did hit lifetime caps & had further treatment '
                     'denied by insurers. They were dropped from plans, '
                     'declared bankruptcy, & died. We are NEVER EVER going '
                     'back to that. #PresidentialDebate2024 #MedTwitter',
             'user': {'avatar': 'https://pbs.twimg.com/profile_images/1352753103926390788/f0sF

### Explore data structure from scrape result

In [6]:
print(type(twitter_hash_tweets))
len(twitter_hash_tweets["tweets"])

<class 'dict'>


10

In [7]:
print(type(twitter_hash_tweets["tweets"]))
print(len(twitter_hash_tweets["tweets"][0]))

<class 'list'>
13


Based on the executed lines of code, the resulting tweets from Nitter stores in the form of python dictionary and the tweets data within the dictionary appears to be python list objects.

In [8]:
# loop through the list object to see what kinds of object are stored in the list
for i in twitter_hash_tweets['tweets'][0]:
    print(i)
print('----------------------------------------')
print(type(twitter_hash_tweets['tweets'][0]))

link
text
user
date
is-retweet
is-pinned
external-link
replying-to
quoted-post
stats
pictures
videos
gifs
----------------------------------------
<class 'dict'>


In [9]:
print(type(twitter_hash_tweets['tweets'][0]['user']))
print(twitter_hash_tweets['tweets'][0]['user'])

<class 'dict'>
{'name': 'Tatiana Prowell, MD', 'username': '@tmprowell', 'profile_id': '1352753103926390788', 'avatar': 'https://pbs.twimg.com/profile_images/1352753103926390788/f0sFNRhL_bigger.jpg'}


In [10]:
print(twitter_hash_tweets['tweets'][0]['user']['username'])

@tmprowell


Now we have knows fully regarding the result of the scrape. Let's extract all the user id

### Extract user id

In [11]:
# Helper function
def extract_element_from_list(init_list:list, element:str) -> list:
    """ 
    The purpose of the function is to extract certain element from the initial list
    Elements included in tweet: (link, text, user, date, is-retweet, is-pinned, external-link, replying-to, quoted-post, stats, pictures, videos, gifs)
    Elements includde in users: (name, username, profile_id, avatar)

    init_list - A list stores a list of dictionary object. 
    elements - A string of the exact element listed above.
    result - A list of detected element
    """
    result = []
    for i in init_list:
        result.append(i[element]) # i must be dictionary 
    return result

In [12]:
tweets = twitter_hash_tweets["tweets"]
users = extract_element_from_list(tweets, 'user')
ids = extract_element_from_list(users, 'username')

In [13]:
for i in ids:
    print(i)

@tmprowell
@Princeofloango
@AfrikanKing_
@savannah_elaine
@PMACampbell
@SISTASinZION
@Garnet_2203
@talleyberrybaby
@LucyBibb
@Captured_Inks


### Extract user profile

In [14]:
# Helper function
def remove_character(init_list:list, character:str) -> list:
    """ 
    The purpose is to remove a character from all elements within the init_list.

    init_list - a list of strings
    charactuer - string value
    result - a list of strings with the character removed. 
    """
    result = []
    for i in init_list:
        if i.count(character) > 0:
            after = i.replace(character, '', 1) # Only remove the first appearance
            result.append(after)
    
    return result

def extract_profile(list_of_ids:list) -> pd.DataFrame:
    """ 
    The purpose is to extract user profile information using their id from a list_of_ids
    
    list_of_ids - a list of ids
    result - a panda dataframe 
    """
    scraper = Nitter(log_level=0, skip_instance_check=False)
    result_list = []
    for id in list_of_ids:
        temp = []
        scrape = scraper.get_profile_info(id, mode='detail')
        if scrape is None:
            continue # Skip all none data to avoid error
        bio = scrape['bio']
        id = scrape['id']
        joined = scrape['joined']
        location = scrape['location']
        name = scrape['name']
        num_follower = scrape['stats']['followers']
        num_following = scrape['stats']['following']
        likes = scrape['stats']['likes']
        media = scrape['stats']['media']
        num_tweets = scrape['stats']['tweets']
        username = scrape['username']
        website = scrape['website']

        temp.append({
            'id': id,
            'username': username,
            'bio': bio,
            'joined_date': joined,
            'location': location,
            'name': name,
            'num_follower': num_follower,
            'num_following': num_following,
            'likes': likes,
            'media': media,
            'num_tweets': num_tweets,
            'website': website
        })

        result_list.extend(temp)
    
    return pd.DataFrame(result_list)

In [15]:
actual_ids = remove_character(ids, '@')
print(actual_ids)

['tmprowell', 'Princeofloango', 'AfrikanKing_', 'savannah_elaine', 'PMACampbell', 'SISTASinZION', 'Garnet_2203', 'talleyberrybaby', 'LucyBibb', 'Captured_Inks']
